In [ ]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams["figure.dpi"] = 100

In [ ]:
# Load the dataset
file_path = 'training_data.csv'
data = pd.read_csv(file_path)
data1=data.copy()
headers=list(data.columns)
# Display the first few rows of the dataframe
data.head()


import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
data_=data.iloc[:,:-1]
minmax.fit(data_)
data = pd.DataFrame(minmax.transform(data_),columns=headers[:-1])
data['increase_stock']=data1['increase_stock']
data.head()


# High demand
The histograms shows the number of observed instances of high demand, given some variable in the x-axis.

In [ ]:
# Filtering the data for only high bike demand (binary label = 1)
data['increase_stock_binary'] = data['increase_stock'].apply(lambda x: 1 if x == 'high_bike_demand' else 0)

high_demand_data = data[data['increase_stock_binary']  == 1]

# Plotting the distribution of each feature for high bike demand
# fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
# axes = axes.flatten()

# fig.suptitle("high demand", y=1, fontsize=30)
# for i, col in enumerate(high_demand_data.columns[:-1]): # excluding the last two columns (original and binary labels)
#     sns.histplot(high_demand_data[col], bins="auto", ax=axes[i], kde=False) 

# plt.tight_layout()
# plt.show()



In [ ]:

plt.figure(figsize=(10, 6))
plt.scatter(range(len(data1['snow'])), data1['snow'])
plt.title('Scatter Plot of Snow')
plt.show()
print(data1['snow'][:10])

# Low demand
The histograms shows the number of observed instances of low demand, given some variable in the x-axis.

In [ ]:
low_demand_data = data[data['increase_stock_binary'] == 0]
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
axes = axes.flatten()

fig.suptitle("low demand", y=1, fontsize=30)
for i, col in enumerate(low_demand_data.columns[:-2]): # excluding the last two columns (original and binary labels)
    sns.histplot(low_demand_data[col], bins="auto", ax=axes[i], kde=False)

plt.tight_layout()
plt.show()

# Plotting low and high demand together
1 = high demand \
0 = low demand

In [ ]:
# Convert the 'increase_stock' column to binary for plotting, 
# data['increase_stock_binary'] = data['increase_stock'].apply(lambda x: 1 if x == 'high_bike_demand' else 0)

# Plotting correlations between inputs and the binary label
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(20, 20))
axes = axes.flatten()

for i, col in enumerate(data.columns[:-1]): # excluding the last column which is the binary label
    sns.boxplot(x='increase_stock_binary', y=col, data=data, ax=axes[i])

plt.tight_layout()
plt.show()

## Observations
1. The increase for demand ishighest in the interval 15:00 - 19:00.
2. Friday is a high demand day.
3. The months has some influence. 
4. Demand is very high when it is not , but slightly higher 
5. 

In [8]:
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split

# 设置测试集大小为20%的原始数据， 设置random_state可以保证每次执行训练时，拆分得到的训练集和测试集都相同， random_state 可以设置成任意整数，只要每次训练使用的值相同即可
X_train, X_test, y_train, y_test=train_test_split(data.iloc[:,:-2],data.iloc[:,-1], test_size=0.2, random_state=0)


gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

# 输出特征重要性得分
print("Feature importance scores:", gb.feature_importances_)

# 选择特征重要性得分排名前2的特征
selected_features = np.array(data.columns)[np.argsort(gb.feature_importances_)[::-1][:9]]
print("Selected features:", selected_features)


Feature importance scores: [4.88109352e-01 2.91003583e-02 1.63549532e-02 6.21649817e-03
 2.86927212e-02 1.46484929e-02 2.45696083e-01 3.06756315e-02
 8.73916584e-02 8.09137786e-03 0.00000000e+00 1.46256297e-04
 2.18554125e-02 1.63485137e-02 6.67269057e-03]
Selected features: ['hour_of_day' 'temp' 'humidity' 'dew' 'day_of_week' 'weekday' 'windspeed'
 'month' 'cloudcover']


In [9]:
select_columns=['hour_of_day', 'temp', 'humidity' ,'dew', 'day_of_week', 'weekday', 'windspeed',
 'month','cloudcover','increase_stock_binary']
data=data[select_columns]

In [12]:
ada = AdaBoostClassifier(random_state=0)
ada.fit(X_train, y_train)




AdaBoostClassifier(random_state=0)

In [10]:
from xgboost.sklearn import XGBClassifier


xgb = XGBClassifier(random_state=0)

xgb.fit(X_train, y_train)



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=0, ...)

In [13]:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=0)
ac=[]
model=ada
for train_index, test_index in kf.split(X_train):
    X_train_kf, X_test_kf = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_kf, y_test_kf = y_train.iloc[train_index], y_train.iloc[test_index]
    model.fit(X_train_kf, y_train_kf)
    ac.append(accuracy_score(y_test_kf, model.predict(X_test_kf)))
print(np.mean(ac))
print("as",accuracy_score(y_test, model.predict(X_test)))
print(ac)

0.8765625


0.84375

In [16]:
from sklearn.model_selection import KFold
kf = KFold(5, shuffle=True, random_state=0)
ac=[]
model=xgb
for train_index, test_index in kf.split(X_train):
    X_train_kf, X_test_kf = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_kf, y_test_kf = y_train.iloc[train_index], y_train.iloc[test_index]
    model.fit(X_train_kf, y_train_kf)
    ac.append(accuracy_score(y_test_kf, model.predict(X_test_kf)))
print(np.mean(ac))
print("as",accuracy_score(y_test, model.predict(X_test)))
print(ac)

0.89921875
as 0.871875
[0.8828125, 0.90234375, 0.91796875, 0.90234375, 0.890625]


### Reference

Freund Y , Schapire R E . A Decision-Theoretic Generalization of On-Line Learning and an Application to Boosting[J]. Journal of Computer and System Sciences, 1997, 55(1):119-139.


In [ ]:
https://blog.csdn.net/weixin_44750583/article/details/100096960